MS-Excel has an option within the Data (in the menu) to obtain data from the web. Used that and then it allows you to save the table on the wikipedia page to xslx/csv. After that just import the file into the notebook and the notebook autogenerates code (Watson removed it) to use pandas to read the table into a pandas dataframe. This likely works well because we were extracting tabular data. But, as the instructions suggest 'beautiful soup" is also a good alternative. Please note that I set Watson up to remove sensitive code cells because the following code accesses a file in my IBM Cloud Object Storage. It includes my credentials.





In [1]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [2]:
df = df.loc[~df['Borough'].str.startswith('Not assigned')]
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [3]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)
df.loc[df['Postcode'] == "M7A"]

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
df.shape

(103, 3)

In [7]:
df2 = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")

In [8]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df2.rename(index=str, columns={'Postal Code': 'Postcode'}, inplace=True)
df3 = pd.merge(df, df2, how='outer', on=['Postcode'])
df3.rename(index=str, columns={'Postcode': 'Postal Code'}, inplace=True)
df3.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [11]:
df3.drop('Postal Code', axis=1, inplace=True)
df3

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476
5,Scarborough,Scarborough Village,43.744734,-79.239476
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [12]:
toronto_data = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
#toronto_data = df3[df3['Borough'].str.contains("Toronto",regex=False)]
toronto_data.head()


,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = 'NJRZIVIZQ4YD30EMS0F0KE1UBSSBQRB5LI2PCEDFPKT1MFG4' # your Foursquare ID
CLIENT_SECRET = 'TTDMDU5FEIVASKL1W3N4NKHEZJCI0NEODHAMQOSS3F3R2UZ4' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: NJRZIVIZQ4YD30EMS0F0KE1UBSSBQRB5LI2PCEDFPKT1MFG4
CLIENT_SECRET:TTDMDU5FEIVASKL1W3N4NKHEZJCI0NEODHAMQOSS3F3R2UZ4


In [16]:
toronto_data.loc[0, 'Neighborhood']
#toronto_data

'The Beaches'

In [17]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=NJRZIVIZQ4YD30EMS0F0KE1UBSSBQRB5LI2PCEDFPKT1MFG4&client_secret=TTDMDU5FEIVASKL1W3N4NKHEZJCI0NEODHAMQOSS3F3R2UZ4&v=20180323&ll=43.67635739999999,-79.2930312&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NJRZIVIZQ4YD30EMS0F0KE1UBSSBQRB5LI2PCEDFPKT1MFG4&client_secret=TTDMDU5FEIVASKL1W3N4NKHEZJCI0NEODHAMQOSS3F3R2UZ4&v=20180323&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e09a4fc1a4b0a001b074f56'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'contact': {},
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.293942087809

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Domino's Pizza,Pizza Place,43.679058,-79.297382


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=NJRZIVIZQ4YD30EMS0F0KE1UBSSBQRB5LI2PCEDFPKT1MFG4&client_secret=TTDMDU5FEIVASKL1W3N4NKHEZJCI0NEODHAMQOSS3F3R2UZ4&v=20180323&ll=43.67635739999999,-79.2930312&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(273, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",7,7,7,7,7,7
Berczy Park,7,7,7,7,7,7
"Brockton,Exhibition Place,Parkdale Village",7,7,7,7,7,7
Business Reply Mail Processing Centre 969 Eastern,7,7,7,7,7,7
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",7,7,7,7,7,7
"Cabbagetown,St. James Town",7,7,7,7,7,7
Central Bay Street,7,7,7,7,7,7
"Chinatown,Grange Park,Kensington Market",7,7,7,7,7,7
Christie,7,7,7,7,7,7


In [27]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 7 unique categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Coffee Shop,Health Food Store,Neighborhood,Other Great Outdoors,Pizza Place,Pub
0,1,0,0,The Beaches,0,0,0
1,0,0,1,The Beaches,0,0,0
2,0,0,0,The Beaches,0,0,1
3,0,0,0,The Beaches,1,0,0
4,0,0,0,The Beaches,0,1,0


In [29]:
toronto_onehot.shape

(273, 7)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Coffee Shop,Health Food Store,Other Great Outdoors,Pizza Place,Pub
0,"Adelaide,King,Richmond",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
1,Berczy Park,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
2,"Brockton,Exhibition Place,Parkdale Village",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
3,Business Reply Mail Processing Centre 969 Eastern,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
5,"Cabbagetown,St. James Town",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
6,Central Bay Street,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
7,"Chinatown,Grange Park,Kensington Market",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
8,Christie,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
9,Church and Wellesley,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857


In [31]:
toronto_grouped.shape

(39, 7)

In [32]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                  venue  freq
0                 Trail  0.14
1           Coffee Shop  0.14
2     Health Food Store  0.14
3  Other Great Outdoors  0.14
4           Pizza Place  0.14


----Berczy Park----
                  venue  freq
0                 Trail  0.14
1           Coffee Shop  0.14
2     Health Food Store  0.14
3  Other Great Outdoors  0.14
4           Pizza Place  0.14


----Brockton,Exhibition Place,Parkdale Village----
                  venue  freq
0                 Trail  0.14
1           Coffee Shop  0.14
2     Health Food Store  0.14
3  Other Great Outdoors  0.14
4           Pizza Place  0.14


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0                 Trail  0.14
1           Coffee Shop  0.14
2     Health Food Store  0.14
3  Other Great Outdoors  0.14
4           Pizza Place  0.14


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
   

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,King,Richmond",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
1,Berczy Park,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
2,"Brockton,Exhibition Place,Parkdale Village",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
3,Business Reply Mail Processing Centre 969 Eastern,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop


In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
#toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
3,East Toronto,Studio District,43.659526,-79.340923,0,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
4,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,The Beaches,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
1,"The Danforth West,Riverdale",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
2,"The Beaches West,India Bazaar",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
3,Studio District,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
4,Lawrence Park,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
5,Davisville North,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
6,North Toronto West,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
7,Davisville,Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
8,"Moore Park,Summerhill East",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Pizza Place,Other Great Outdoors,Health Food Store,Coffee Shop


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
